In [57]:
type ('a, 'b) result = Ok of 'a | Err of 'b

type request = { name : string; email : string }

let validateInput input =
    if input.name = "" then Err "Name must not be blank"
    else if input.email = "" then Err "Email must not be blank"
    else Ok input;;

let validateName input =
    if input.name = "" then Err "Name is blank"
    else Ok input
    
let validateNameLength input =
    if String.length input.name > 20 then Err "Name is too long"
    else Ok input
    
let validateEmail input =
    if input.email = "" then Err "Email is blank"
    else Ok input
    
let bind f input =
    match input with
    | Ok x -> f x
    | Err e -> Err e;;

type ('a, 'b) result = Ok of 'a | Err of 'b


type request = { name : string; email : string; }


val validateInput : request -> (request, string) result = <fun>


val validateName : request -> (request, string) result = <fun>


val validateNameLength : request -> (request, string) result = <fun>


val validateEmail : request -> (request, string) result = <fun>


val bind : ('a -> ('b, 'c) result) -> ('a, 'c) result -> ('b, 'c) result =
  <fun>


In [58]:
let badReq = { name = ""; email = ""};;
let goodReq = { name = "Foo"; email = "foo@bar.COM " };;
let (>>) f g x = g (f x);;

let validateReq = validateName >> bind validateNameLength >> bind validateEmail;;
validateReq badReq;;

let (>>=) x f = bind f x;;
        
badReq 
|> validateName
>>= validateNameLength ;;

val badReq : request = {name = ""; email = ""}


val goodReq : request = {name = "Foo"; email = "foo@bar.COM "}


val ( >> ) : ('a -> 'b) -> ('b -> 'c) -> 'a -> 'c = <fun>


val validateReq : request -> (request, string) result = <fun>


- : (request, string) result = Err "Name is blank"


val ( >>= ) : ('a, 'b) result -> ('a -> ('c, 'b) result) -> ('c, 'b) result =
  <fun>


- : (request, string) result = Err "Name is blank"


In [59]:
let (>==>) f1 f2 x =
 match f1 x with
 | Ok o -> f2 o
 | Err e -> Err e;;
 
 let validateReq =  validateName >==> validateNameLength >==>  validateEmail;;
 validateReq badReq;;
 
let (=>>= ) f1 f2 x =
    f1 x >>= f2 ;;
    
let validateReq =  validateName =>>= validateNameLength =>>=  validateEmail;;
 validateReq badReq;;

val ( >==> ) :
  ('a -> ('b, 'c) result) -> ('b -> ('d, 'c) result) -> 'a -> ('d, 'c) result =
  <fun>


val validateReq : request -> (request, string) result = <fun>


- : (request, string) result = Err "Name is blank"


val ( =>>= ) :
  ('a -> ('b, 'c) result) -> ('b -> ('d, 'c) result) -> 'a -> ('d, 'c) result =
  <fun>


val validateReq : request -> (request, string) result = <fun>


- : (request, string) result = Err "Name is blank"


In [61]:
let cannonicalizeEmail input =
    {input with email = String.((trim >> lowercase_ascii) input.email) };;
    
let switch f x = Ok (f x);;

let map f x =
    match x with
    | Ok o -> Ok (f o)
    | Err e -> Err e ;;

let tee f x =
    f x |> ignore;
    x ;;
    
tee (fun x -> x + 1) 10;;

let updateDB x = ();;

let tryCatch f x =
    try 
        Ok (f x)
    with
    | _ -> Err "Failed";;

let doubleMap okF errF input =
    match input with
    | Ok o -> Ok (okF o)
    | Err e -> Err (errF e) ;;

let log input =
    let ok x = print_string "ok\n" in
    let err x = print_string x in
    doubleMap ok err input;;

let ok x = Ok x;;
let err x = Err x;;

let combine okF errF f1 f2 x =
    match f1 x, f2 x with
    | Ok x1, Ok x2 -> Ok (okF x1 x2)
    | Ok _, Err e -> Err e
    | Err e, Ok _ -> Err e
    | Err e1, Err e2 -> Err (errF e1 e2);;

let (<&>) f1 f2 =
    let ok r1 r2 = r1 in
    let err e1 e2 = e1 ^ ":" ^ e2 in
    combine ok err f1 f2;;

let useCase =  
     validateName 
     <&> validateNameLength 
     <&> validateEmail
     <&> switch cannonicalizeEmail (* or >> map cannonicalizeEmail *)
     <&> tryCatch (tee updateDB)
     >> log;;
     
useCase goodReq;;
useCase badReq;;
print_newline();;

validateName 
     <&> validateNameLength ;;

val cannonicalizeEmail : request -> request = <fun>


val switch : ('a -> 'b) -> 'a -> ('b, 'c) result = <fun>


val map : ('a -> 'b) -> ('a, 'c) result -> ('b, 'c) result = <fun>


val tee : ('a -> 'b) -> 'a -> 'a = <fun>


- : int = 10


...

- : (unit, unit) result = Ok ()


- : (unit, unit) result = Err ()


ok
Name is blank:Email is blank


- : unit = ()


- : request -> (request, string) result = <fun>
